# NEU-PCA: MNIST
- Designed and Coded by: [Anastasis Kratsios](https://people.math.ethz.ch/~kratsioa/).
- Some Elements of the PCA analysis are forked from [this repo](https://github.com/radmerti/MVA2-PCA/blob/master/YieldCurvePCA.ipynb).

## Parameters

In [1]:
PCA_Rank = 2

## Imports

In [2]:
# First Round Initializations (Global Level) #
#============================================#
# Load Dependances and makes path(s)
exec(open('Initializations_Dump.py').read())
# Load Hyper( and meta) parameter(s)
exec(open('HyperParameter_Grid.py').read())
# %run Helper_Functions.ipynb
exec(open('Helper_Functions.py').read())
# Load Models
# %run Architecture_Builder.ipynb
exec(open('Architecture_Builder.py').read())
# Initialize "First Run Mode"
First_run = True

Using TensorFlow backend.


Parameter Grids Build and Loaded!
Complete NEU-Structure Building Procedure!!!
Complete NEU-Structure Building Procedure!!!
Complete NEU-ffNN Training Procedure!!!
Deep Feature Builder - Ready
Complete NEU-ffNN Training Procedure!!!
Complete NEU-ffNN Training Procedure!!!
Complete NEU-ffNN (Fully Coupled) Training Procedure!!!
Complete NEU-ffNN Training Procedure!!!
Complete NEU-ffNN Training Procedure!!!
Complete NEU-PCA Training Procedure!!!


In [3]:
import pylab as plt
import numpy as np
import seaborn as sns; sns.set()

import sklearn
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import decomposition
import scipy

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Layer, InputSpec
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import regularizers, activations, initializers, constraints, Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.constraints import UnitNorm, Constraint

from numpy.random import seed

### Fix Seeds for Reproducability

In [4]:
# Numpy
np.random.seed(2020)
# Tensorflow
tf.random.set_seed(2020)
# Python's Seed
random.seed(2020)

## Load Data

In [5]:
from tensorflow.keras.datasets import mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784) / 255
X_test = X_test.reshape(10000, 784) / 255

#### Pre-Process Data

In [6]:
# Initialize Scaler
scaler = MinMaxScaler()
scaler.fit(X_train)

# Train Scaler
X_train_scaled = scaler.transform(X_train)
# Map to Test Set
X_test_scaled = scaler.transform(X_test)

In [7]:
# # ## TEMPS!!
is_visuallty_verbose = True
# First_run = False

---

# Benchark(s)
---

## Get PCAs

In [8]:
# Reconstruct Training Data
Zpca,Zpca_test,Rpca,Rpca_test = get_PCAs(X_train_scaled=X_train_scaled,
                                         X_test_scaled=X_test_scaled,
                                         PCA_Rank=PCA_Rank)

#### Get Reconstruction Result(s)

In [9]:
# Get Results #
#-------------#
## Compute
PCA_Reconstruction_results = reporter_array(Rpca,Rpca_test,X_train,X_test)
## Organize
### Train
Performance_Results_train = pd.DataFrame(PCA_Reconstruction_results['Train'],index=PCA_Reconstruction_results.index)
Performance_Results_train.columns=['PCA']
### Test
Performance_Results_test = pd.DataFrame(PCA_Reconstruction_results['Test'],index=PCA_Reconstruction_results.index)
Performance_Results_test.columns=['PCA']

# Update Total Results #
#----------------------#
# NA

# Save Results #
#--------------#
Performance_Results_train.to_latex('outputs/tables/MNIST_Performance_train.txt')
Performance_Results_test.to_latex('outputs/tables/MNIST_Performance_test.txt')

## Get (ReLU) Auto-Encoder

In [ ]:
AE_Reconstructed_train, AE_Reconstructed_test, AE_Factors_train = build_autoencoder(CV_folds,
                                                                                    n_jobs,
                                                                                    n_iter,
                                                                                    X_train_scaled,
                                                                                    X_train,
                                                                                    X_test_scaled,
                                                                                    PCA_Rank)

Begin autoencoder Training
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 out of   2 | elapsed:  1.4min remaining:    0.0s
[Parallel(n_jobs=3)]: Done   2 out of   2 | elapsed:  1.4min finished


We use a DNN of depth: 6
Epoch 1/4
 212/3750 [>.............................] - ETA: 17s - loss: 0.0837 - accuracy: 0.0088

#### Get Reconstruction Result(s)

In [ ]:
# Get Results #
#-------------#
## Compute
AE_Reconstruction_results = reporter_array(AE_Reconstructed_train,AE_Reconstructed_test,X_train,X_test)
## Organize
### Train
AE_Performance_Results_train = pd.DataFrame(AE_Reconstruction_results['Train'],index=AE_Reconstruction_results.index)
AE_Performance_Results_train.columns=['AE']
### Test
AE_Performance_Results_test = pd.DataFrame(AE_Reconstruction_results['Test'],index=AE_Reconstruction_results.index)
AE_Performance_Results_test.columns=['AE']

# Update Total Results #
#----------------------#
Performance_Results_train = pd.concat([Performance_Results_train,AE_Performance_Results_train],axis=1)
Performance_Results_test = pd.concat([Performance_Results_test,AE_Performance_Results_test],axis=1)

# Save Results #
#--------------#
Performance_Results_train.to_latex('outputs/tables/MNIST_Performance_train.txt')
Performance_Results_test.to_latex('outputs/tables/MNIST_Performance_test.txt')

# Visualize Results

### Feature Space(s)

In [ ]:
plt.figure(figsize=(16,16))
plt.subplot(121)
plt.title('PCA')
plt.scatter(Zpca[:5000,0], Zpca[:5000,1], c=Y_train[:5000], s=8, cmap='tab10')
plt.gca().get_xaxis().set_ticklabels([])
plt.gca().get_yaxis().set_ticklabels([])

plt.subplot(122)
plt.title('Autoencoder')
plt.scatter(AE_Factors_train[:5000,0], AE_Factors_train[:5000,1], c=Y_train[:5000], s=8, cmap='tab10')
plt.gca().get_xaxis().set_ticklabels([])
plt.gca().get_yaxis().set_ticklabels([])

plt.tight_layout()

# Save Results
plt.savefig('outputs/plotsANDfigures/Results_Visualization_MNIST.pdf')

## Reconstruction(s)

In [ ]:
plt.figure(figsize=(9,3))
toPlot = (X_train, Rpca, AE_Reconstructed_train)
for i in range(10):
    for j in range(3):
        ax = plt.subplot(3, 10, 10*j+i+1)
        plt.imshow(toPlot[j][i,:].reshape(28,28), interpolation="nearest", 
                   vmin=0, vmax=1)
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

plt.tight_layout()

---

## Numerical Summary

#### Testing Results

In [ ]:
print(np.round(Performance_Results_test,4))
Performance_Results_test

#### Training Results

In [ ]:
print(np.round(Performance_Results_train,4))
Performance_Results_train

---
# Fin
---